In [ ]:
!pip install transformers
!pip install pyspark
!pip install sparkdl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=5a8bfb01296108a1ee3670e0c4d6090012307a8ac8bac889aa685c757f23040a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import VectorUDT, Vectors
from transformers import BertModel
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# إعداد جلسة Spark
spark = SparkSession.builder \
    .appName("Text Classification") \
    .getOrCreate()

# تعيين المسارات للمجلدين
fake_news_folder = "/content/drive/MyDrive/dataset/fake"
real_news_folder = "/content/drive/MyDrive/dataset/real"

# قراءة النصوص الكاذبة
texts = []
labels = []
for filename in os.listdir(fake_news_folder):
    file_path = os.path.join(fake_news_folder, filename)
    with open(file_path, "r", encoding='cp1256') as file:
        text = file.read()
        texts.append(text)
        labels.append(0)

# قراءة النصوص الصادقة
for filename in os.listdir(real_news_folder):
    file_path = os.path.join(real_news_folder, filename)
    with open(file_path, "r", encoding='cp1256') as file:
        text = file.read()
        texts.append(text)
        labels.append(1)



In [ ]:
# تحميل المحددات من النموذج
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv2')

# تحويل النصوص إلى تمثيلات BERT
encoded_texts = []
for news_text in texts:
    inputs = tokenizer.encode_plus(
        news_text,
        None,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    encoded_texts.append(inputs['input_ids'])

# تحويل التمثيلات إلى تنسورفلو
features = tf.concat(encoded_texts, axis=0)

# تحويل التسميات إلى تنسورفلو
labels = tf.convert_to_tensor(labels)

# تحويل الأبعاد
features = tf.reshape(features, (-1, 128))

# تحويل features و labels إلى NumPy arrays
features = features.numpy()
labels = labels.numpy()

# خلط البيانات
from sklearn.utils import shuffle

features, labels = shuffle(features, labels, random_state=42)

# تقسيم البيانات إلى بيانات التدريب والاختبار
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

# تحويل البيانات إلى DataFrame في Spark
train_df = spark.createDataFrame(zip(train_features.tolist(), train_labels.tolist()), ["text", "label"])
test_df = spark.createDataFrame(zip(test_features.tolist(), test_labels.tolist()), ["text", "label"])

# تحويل النص إلى نوع Vector
to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
train_df = train_df.withColumn("text", to_vector("text"))
test_df = test_df.withColumn("text", to_vector("text"))


In [ ]:
# تعديل النموذج إلى LSTM
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(128, 1)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


# تحويل DataFrame إلى RDD للتدريب مع TensorFlow
train_rdd = train_df.rdd.map(lambda row: (row['text'].toArray(), int(row['label'])))
train_data = train_rdd.map(lambda x: (x[0].tolist(), x[1])).toDF(['text', 'label'])



In [ ]:
# تحويل بيانات التدريب إلى مصفوفات NumPy
train_texts = train_data.select('text').rdd.map(lambda x: x[0]).collect()
train_labels = train_data.select('label').rdd.map(lambda x: x[0]).collect()

# تدريب النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_texts, train_labels, epochs=100)


Epoch 1/100
51/51 [==============================] - 15s 223ms/step - loss: 0.7020 - accuracy: 0.4795
Epoch 2/100
51/51 [==============================] - 10s 200ms/step - loss: 0.6934 - accuracy: 0.5112
Epoch 3/100
51/51 [==============================] - 10s 184ms/step - loss: 0.6910 - accuracy: 0.5149
Epoch 4/100
51/51 [==============================] - 13s 254ms/step - loss: 0.6924 - accuracy: 0.5087
Epoch 5/100
51/51 [==============================] - 10s 199ms/step - loss: 0.6840 - accuracy: 0.5386
Epoch 6/100
51/51 [==============================] - 9s 186ms/step - loss: 0.6854 - accuracy: 0.5224
Epoch 7/100
51/51 [==============================] - 9s 166ms/step - loss: 0.6774 - accuracy: 0.5709
Epoch 8/100
51/51 [==============================] - 13s 257ms/step - loss: 0.6688 - accuracy: 0.5808
Epoch 9/100
51/51 [==============================] - 11s 216ms/step - loss: 0.6679 - accuracy: 0.5889
Epoch 10/100
51/51 [==============================] - 11s 217ms/step - loss: 0.6752 

In [ ]:
import numpy as np

# تحويل بيانات الاختبار إلى مصفوفات NumPy
test_texts = np.array(test_df.select('text').collect())
test_labels = np.array(test_df.select('label').collect())

# تحويل البيانات إلى الأبعاد المطلوبة
test_texts = np.squeeze(test_texts, axis=1)
test_labels = np.squeeze(test_labels, axis=1)

# تقييم النموذج على بيانات الاختبار
test_metrics = model.evaluate(test_texts, test_labels)

# طباعة المقاييس
print("Test Data Metrics:")
print("Loss:", test_metrics[0])
print("Accuracy:", test_metrics[1])

from sklearn.metrics import precision_score, recall_score, f1_score

# استخراج التوقعات من النموذج
test_predictions = model.predict(test_texts)
test_predictions = np.round(test_predictions).astype(int)

# حساب المقاييس الإضافية
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

# طباعة المقاييس الإضافية
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


13/13 [==============================] - 4s 115ms/step - loss: 0.3051 - accuracy: 0.8731
Test Data Metrics:
Loss: 0.30514007806777954
Accuracy: 0.8731343150138855
13/13 [==============================] - 3s 109ms/step
Precision: 0.8744186046511628
Recall: 0.8867924528301887
F1 Score: 0.8805620608899298


In [7]:
model.save("LSTM")